In [32]:
import numpy as np
import time
import math
import random
import gym
from typing import Tuple
import matplotlib.pyplot as plt
from IPython import display

In [25]:
# CartPole-V1 Enviornment Setup

In [26]:
env = gym.make('CartPole-v1')

In [27]:
# Enviornment Documentation
?env.env

In [28]:
# Visualization

In [33]:
policy = lambda obs: 0
obs = env.reset()
for i in range(25):
    plt.imshow(env.render(mode='rgb_array'))
    display.display(plt.gcf())    
    display.clear_output(wait=True)
    env.step(env.action_space.sample()) # take a random action

env.close()

ImportError: 
    Cannot import pyglet.
    HINT: you can install pyglet directly via 'pip install pyglet'.
    But if you really just want to install all Gym dependencies and not have to think about it,
    'pip install -e .[all]' or 'pip install gym[all]' will do it.
    